# <center>Implementing LDA in Python</center>

<center>Dr. W.J.B. Mattingly</center>

<center>Smithsonian Data Science Lab and United States Holocaust Memorial Museum</center>

<center>February 2021</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [3]:
stopwords = stopwords.words("english")

In [4]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
data = load_data("data/ushmm_dn.json")["texts"]

print (data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [21]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [22]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [24]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 8), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [26]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


## Vizualizing the Data

In [27]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.025607  0.293203       1        1  21.721140
2      0.148460  0.331648       2        1  20.611230
24     0.071520  0.359273       3        1  12.493062
15     0.032683  0.396110       4        1  11.556512
9      0.248335  0.354148       5        1  10.847792
19     0.209884  0.378408       6        1   7.268443
12     0.140075  0.376969       7        1   7.107453
21    -0.138464  0.329285       8        1   6.490240
18    -0.068963  0.366136       9        1   1.662169
6      0.251815 -0.001492      10        1   0.223031
25    -0.012343 -0.177815      11        1   0.013639
17    -0.047814 -0.158177      12        1   0.000304
16    -0.047815 -0.158182      13        1   0.000303
22    -0.047816 -0.158184      14        1   0.000300
8     -0.047816 -0.158186      15        1   0.000300
14    -0.047819 -0.158194      16        1   0.000292
23    -0.047821 -0.158202      17        1   0.000286
27    -0.047822 -0.158206      18        1   0.000284
28    -0.047822 -0.158205      19        1   0.000282
4     -0.047822 -0.158206      20        1   0.000280
7     -0.047824 -0.158211      21        1   0.000275
0     -0.047824 -0.158211      22        1   0.000272
29    -0.047824 -0.158211      23        1   0.000272
11    -0.047824 -0.158212      24        1   0.000268
10    -0.047824 -0.158213      25        1   0.000266
20    -0.047824 -0.158213      26        1   0.000265
1     -0.047825 -0.158214      27        1   0.000261
13    -0.047825 -0.158216      28        1   0.000261
5     -0.047825 -0.158215      29        1   0.000260
26    -0.047825 -0.158216      30        1   0.000260, topic_info=       Term          Freq         Total Category  logprob  loglift
904   would   8150.000000   8150.000000  Default  30.0000  30.0000
446    know  21219.000000  21219.000000  Default  29.0000  29.0000
344      go  24522.000000  24522.000000  Default  28.0000  28.0000
695     say  14619.000000  14619.000000  Default  27.0000  27.0000
166    come  14416.000000  14416.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
867    want      0.000167   5565.190503  Topic30  -9.6361  -4.4641
298  father      0.000167   3978.360804  Topic30  -9.6361  -4.1285
488     man      0.000167   3496.535151  Topic30  -9.6361  -3.9994
469  little      0.000167   4627.312828  Topic30  -9.6361  -4.2796
486    make      0.000167   5055.998798  Topic30  -9.6361  -4.3682

[2943 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.148470   able
0         2  0.142320   able
0         3  0.166040   able
0         4  0.327688   able
0         5  0.013178   able
...     ...       ...    ...
3954      2  0.027790  zloty
3954      3  0.833693  zloty
3954      7  0.083369  zloty
3954      8  0.027790  zloty
8537     10  0.910631   zoom

[2774 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 25, 16, 10, 20, 13, 22, 19, 7, 26, 18, 17, 23, 9, 15, 24, 28, 29, 5, 8, 1, 30, 12, 11, 21, 2, 14, 6, 27])